<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/excercise/Design_of_a_TEG_dehydration_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example 18.3 from Cambell

Calculate the circulation rate of 98.7 wt% of lean TEG needed to dry 1 MSm3/day of gas at 70 bara and $40 ^{\circ}C$ in a six tray absorber (1.5 theoretical stages) to achieve a water content of 117 kg/MSm3. THe inlet gas is saturated with water (1100 kg water/MSm3).

## Calculate the circulation rate using the equiations from Campbell book

The following equiations will be used in the calculations:

$E_a=\frac{W_{N+1}-W_1}{W_{N+1}-W_0} = \frac{A^{N+1}-A}{A^{N+1}-1}$ (18.2) 

Where $W_N+1$ is the water content in (kg water/MSm3) in the feed gas (1100 kg/MSm3). $W_1$ is the water content in the dehydrated gas (117 kg/MSm3). A is the absorption factor calculates as A=L/VK. Where V and L is gas and liquid flow rate in mol/hr. K is the equilibrium constant.

$W_0$ is given by raults law and is calculated from: 

$W_0=W\cdot \gamma_{water} \cdot x_{water} $ (18.4)

Where $\gamma_{water}$ is the activity coefficient of water in lean TEG, and $x_{water}$ is the mol fraction of water in the lean TEG. From figure 18.7 we read at $40 ^{\circ}C$ and 98.7 wt% TEG: $\gamma_{water}=0.66$

$x_{water}=\frac{\frac{100-X_{gl}}{18}}{\frac{100-X_{gl}}{18}+\frac{X_gl}{150}}$

THe equilibrium constant for water is calculated from:

$K_{water}=\frac{y_{water}}{x_{water}}=\gamma \cdot \frac{P_{sat_water}}{P} = \gamma_{water} \cdot W_{sat}/B$

Where B is kg gas/MSm3 and is  calculated assuming a molar mass of about 20.0 gr/mol of gas.

$B=\frac{n}{V} \cdot M = \frac{P}{RT} \cdot M = \frac{101325}{8.314*288.15} \cdot 20.0 = 761000 \frac{kg}{MSm^3}$


In [0]:
GasFlowRate = 1.0 # MSm3/day
WNp1 = 1100.0 # kg water/MSm^3 (saturated at inlet)
W1 = 117.0 # kg water/MSm3
Np1 = 1.5 # Number of equilibrium stages
wtLeanTEG = 98.7
print("Number of equilibrium stages ",Np1)
x0 = (100.0-wtLeanTEG)/18.0/((100.0-wtLeanTEG)/18.0+wtLeanTEG/150.0) # mole fraction water in lean TEG
print("x0 " , x0, " [mol water/mol tot]")

gamma = 0.66 #activity coefficient of water

W0 = WNp1*gamma*x0 #water content in equilibrium with lean TEG
print("W0 " , W0," kg/MSm3")

Ea = (WNp1-W1)/(WNp1-W0) #absorption efficiency calulated from relative water removal from gas
print("Ea " , Ea," -")

## setting up a function to solve A 
def func(A):
        return (pow(A,Np1+1.0)-A)/(pow(A,Np1+1.0)-1.0)-Ea

from scipy.optimize import fsolve
solveA = fsolve(func, 0.1) # solve absorption factor using 0.1 as initial guess
A = solveA[0]
print('A ', A)
Ea2 = (pow(A,Np1+1.0)-A)/(pow(A,Np1+1.0)-1.0)
print("Ea2 " , Ea2," -")


VSm3hr = GasFlowRate*1e6/24.0 # MSm3/hr
molperm3 = 101325.0/8.314/288.15  #mol/Sm3
Vmolhr = VSm3hr*molperm3/1000.0 # mol/hr
print("V_mol/hr " , Vmolhr," kmol/hr")

B = 101325/(8.314*(273.15+40.0))*1e6 * 20.0/1000.0 #calculate B assuming molar mass of gas of 20.0 gr/mol
# in this case we use B from book
B = 761000.0

K = WNp1*gamma/B # Calculating equilibrium constant
print("K equilibrium constant water " , K," [-]")
Lmolhr =  A*Vmolhr*K
print("TEG mol/hr " , Lmolhr," kmol/hr")

molecularW = (1.0-x0)*150.0 + x0*18.0
print("molecularW" , molecularW," gr/mol")
massRateTEG = molecularW * Lmolhr
print("mass rate glycol " , massRateTEG," kg TEG/hr")

densityTEG = 1.12 #kg/liter


absRateWater = (WNp1-W1)/24.0
print("absorption rate water " , absRateWater," kg water/hr")

circRatio = massRateTEG/densityTEG/absRateWater
print("circulation ratio TEG/water " , circRatio," liter TEG/kg water")


Number of equilibrium stages  1.5
x0  0.09890444309190485  [mol water/mol tot]
W0  71.80462568472292  kg/MSm3
Ea  0.9560439820638419  -
A  7.312899327464227
Ea2  0.9560439820638403  -
V_mol/hr  1762.2873614416887  kmol/hr
K equilibrium constant water  0.0009540078843626807  [-]
TEG mol/hr  12.294709886685112  kmol/hr
molecularW 136.94461351186857  gr/mol
mass rate glycol  1683.694293672642  kg TEG/hr
absorption rate water  40.958333333333336  kg water/hr
circulation ratio TEG/water  36.703116414895554  liter TEG/kg water


# HYSYS calculation
See HYSYS simulation file